In [36]:
import pandas as pd
import numpy as np
import os
import glob
import pickle

pd.set_option('max_colwidth', 800)

from datetime import datetime
from datetime import timedelta
from datetime import date

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import contractions
import nltk

# import spacy

# nltk.download('stopwords')

import re                                  
import string                             
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer 

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import math
#import xlsxwriter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
wnl = nltk.stem.WordNetLemmatizer()

# loading up the language model: English
# nlp = spacy.load('en')

# from sklearn.feature_extraction import text
# stop = text.ENGLISH_STOP_WORDS
stop = ['etc', 's', 't', 'ie', 'amp']

lemmatizer = nltk.stem.WordNetLemmatizer()

import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt 

# Load Sales Data

In [38]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "../data_collection/sales_records/Andre_sales_records_1-50/*.csv"))
csv_files2 = glob.glob(os.path.join(path, "../data_collection/sales_records/Andre_sales_records_100-150/*.csv"))

file_list = []

for file in csv_files:
    if file.endswith('.csv'):
        df = pd.read_csv(file,sep=",")
        df['filename'] = file
        file_list.append(df)
        
for file2 in csv_files2:
    if file2.endswith('.csv'):
        df2 = pd.read_csv(file2,sep=",")
        df2['filename'] = file2
        file_list.append(df2)

sales_df = pd.concat(file_list, ignore_index=True)

sales_df['NFTcollection'] = sales_df.filename.str.split('/').str[6]
sales_df['NFTcollection'] = sales_df.NFTcollection.str.split('-').str[0]
sales_df['NFTcollection'] = sales_df['NFTcollection'].str.strip()
sales_df = sales_df[['DateTime', 'Sales (USD)', 'NFTcollection']]

# Twitter Data

In [39]:
tweet_df = pd.read_csv('../data_collection/twitter/frame_1-50_100-150col.csv', index_col=None, header=0)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
tweet_df.head()

In [27]:
list_100_nfts = list(tweet_df.columns.values)

list_100_nfts.remove('text')
list_100_nfts.remove('created_at')
list_100_nfts.remove('retweets')
list_100_nfts.remove('replies')
list_100_nfts.remove('likes')
list_100_nfts.remove('quote_count')

#set(sales_df.NFTcollection.unique()) ^ set(list_100_nfts)

# Google Trends Data

In [28]:
trends_df = pd.read_csv('../data_collection/trends/data/daily_interest.csv', index_col=None, header=0)
trends_df = trends_df.rename(columns={"date": "created_at"})
trends_df = trends_df.drop_duplicates()
trends_df = trends_df.fillna(0)

# Functions

In [29]:
def clean_tweets(df):
    #remove special characters
    df['Tweet_nospecial'] = df["text"].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
    #Remove hyperlinks
    df['Tweet_noURL'] = df['Tweet_nospecial'].str.replace('http\S+','')
    #Remove mentions
    df['Tweet_noMentions'] = df['Tweet_noURL'].str.replace(r'@[A-Za-z0-9_]+', '', regex=True)
    #Expand contractions
    df["Tweet_expanded"]=df['Tweet_noMentions'].apply(lambda x: [contractions.fix(word) for word in x.split()])
    df['Tweet_expanded'] = [' '.join(map(str, l)) for l in df['Tweet_expanded']]
    #remove newline
    df["Tweet_noNewLine"]= df['Tweet_expanded'].replace(r'\\n', ' ', regex=True)
    #Remove punctuation
    df["Tweet_noPunctuation"] = df['Tweet_noNewLine'].str.replace('[^\w\s]','')
    df["Tweet_noPunctuation"] = df['Tweet_noPunctuation'].str.replace('[_]','')
    #Remove numbers
    df["Tweet_noNumbers"] = df['Tweet_noPunctuation'].str.replace(r'[0-9]',' ')
    #Make lowercase
    df['Tweet_lowercase'] = df.Tweet_noNumbers.str.lower()
    #remove stopwords
    df['Tweet_noStopwords'] = df['Tweet_lowercase'].apply(lambda x: [item for item in x.split() if item not in stop])
    #lemmatize words
    #df['Tweet_lemmatized'] = df['Tweet_noStopwords'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
    #subset columns
    df = df[['text','created_at', 'retweets', 'Tweet_noStopwords']].copy()
    df['Tweet_noStopwords'] = [' '.join(map(str, l)) for l in df['Tweet_noStopwords']]
    
    return df

In [30]:
def sentimentScore(Tweet, df):
    analyzer = SentimentIntensityAnalyzer()
    score_values = []
    for tweet in df['Tweet_noStopwords']:
        vs1 = analyzer.polarity_scores(tweet)
#         print("Score: " + str(vs1))
        score_values.append(vs1)
    return score_values

In [31]:
def calc_vader_sentiment(row):
    if row['retweets'] == 0:
        return row['compound']
    else:
        return row['compound']*row['retweets']

In [32]:
def mergeTweetSales(nft_col):
    #prep tweet data per nft collection
    nft_tweet_df = tweet_df[tweet_df[nft_col] == 1][['text', 'created_at', 'retweets']].reset_index(drop=True)
    nft_tweet_df = clean_tweets(nft_tweet_df)
    nft_tweet_df['created_at'] = nft_tweet_df.created_at.str.split(' ').str[0]
    sentiment_scores = pd.DataFrame(sentimentScore(nft_tweet_df['Tweet_noStopwords'], nft_tweet_df))
    nft_tweet_df = pd.merge(nft_tweet_df, sentiment_scores, left_index=True, right_index=True)
    nft_tweet_df['sentiment_vader_score'] = nft_tweet_df.apply(calc_vader_sentiment, axis=1)
    nft_tweet_df_daily_mean=(nft_tweet_df.groupby(nft_tweet_df.created_at).mean()).reset_index()
    nft_tweet_df_daily_count = nft_tweet_df.groupby('created_at')['text'].count().to_frame(name = 'num_tweets').reset_index()
    nft_tweet_df_mean_count = pd.merge(nft_tweet_df_daily_mean, nft_tweet_df_daily_count, on='created_at')
    nft_tweet_df_mean_count['final_vader_score'] = nft_tweet_df_mean_count['sentiment_vader_score']*nft_tweet_df_mean_count['num_tweets']
    nft_sentiment_df = nft_tweet_df_mean_count[['created_at', 'final_vader_score', 'num_tweets']]
    
    #prep sales data per nft collection
    nft_sales_df = sales_df[sales_df['NFTcollection'] == nft_col]
    nft_sales_df['DateTime'] = nft_sales_df.DateTime.str.split(' ').str[0]
    nft_sales_df = nft_sales_df.rename(columns={"DateTime": "created_at"})
    nft_sales_df = nft_sales_df.rename(columns={"Sales (USD)": "Sales"})
    scaler = StandardScaler()
    nft_sales_df['prices_scaled'] = scaler.fit_transform(nft_sales_df[['Sales']])
    nft_sales_df = nft_sales_df[['created_at', 'prices_scaled']]
    
    #merge tweet and sales data
    nft_merged = pd.merge(nft_sentiment_df, nft_sales_df, on='created_at')
    nft_merged['previous_day_sentiment'] = nft_merged['final_vader_score'].shift(1)
    
    #merge data with Google trends data
    nft_trends_df = trends_df[trends_df['collection_name'] == nft_col]
    nft_merged_final = pd.merge(nft_merged, nft_trends_df, how = 'left', on='created_at')
    
    nft_merged_final = nft_merged_final[['created_at', 'final_vader_score', 'prices_scaled', 'previous_day_sentiment', 'trends', 'trends_previous_day', 'trends_two_days_prior', 'num_tweets']]
    nft_merged_final = nft_merged_final.dropna()
        
    return nft_merged_final

In [33]:
finaldf = mergeTweetSales(sales_df.NFTcollection.unique()[0])

<ipython-input-29-fe888b4778db>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet_noURL'] = df['Tweet_nospecial'].str.replace('http\S+','')
<ipython-input-29-fe888b4778db>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noNewLine'].str.replace('[^\w\s]','')
<ipython-input-29-fe888b4778db>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noPunctuation'].str.replace('[_]','')
<ipython-input-29-fe888b4778db>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noNumbers"] = df['Tweet_noPunctuation'].str.replace(r'[0-9]',' ')
<ipython-input-32-3b2f1c55a8d5>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [34]:
l = []

finaldf = mergeTweetSales(sales_df.NFTcollection.unique()[0])
finaldf['collection_name'] = sales_df.NFTcollection.unique()[0]
for x in sales_df.NFTcollection.unique()[1:]: 
    try:
        temp_df = mergeTweetSales(x)
        temp_df['collection_name'] = x
        finaldf = finaldf.append(temp_df)
    except:
        pass

<ipython-input-29-fe888b4778db>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet_noURL'] = df['Tweet_nospecial'].str.replace('http\S+','')
<ipython-input-29-fe888b4778db>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noNewLine'].str.replace('[^\w\s]','')
<ipython-input-29-fe888b4778db>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noPunctuation'].str.replace('[_]','')
<ipython-input-29-fe888b4778db>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noNumbers"] = df['Tweet_noPunctuation'].str.replace(r'[0-9]',' ')
<ipython-input-32-3b2f1c55a8d5>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

<ipython-input-32-3b2f1c55a8d5>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nft_sales_df['DateTime'] = nft_sales_df.DateTime.str.split(' ').str[0]
<ipython-input-29-fe888b4778db>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet_noURL'] = df['Tweet_nospecial'].str.replace('http\S+','')
<ipython-input-29-fe888b4778db>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noNewLine'].str.replace('[^\w\s]','')
<ipython-input-29-fe888b4778db>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noPunctuation'].str.replac

<ipython-input-32-3b2f1c55a8d5>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nft_sales_df['DateTime'] = nft_sales_df.DateTime.str.split(' ').str[0]
<ipython-input-29-fe888b4778db>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet_noURL'] = df['Tweet_nospecial'].str.replace('http\S+','')
<ipython-input-29-fe888b4778db>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noNewLine'].str.replace('[^\w\s]','')
<ipython-input-29-fe888b4778db>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noPunctuation'].str.replac

<ipython-input-32-3b2f1c55a8d5>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nft_sales_df['DateTime'] = nft_sales_df.DateTime.str.split(' ').str[0]
<ipython-input-29-fe888b4778db>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet_noURL'] = df['Tweet_nospecial'].str.replace('http\S+','')
<ipython-input-29-fe888b4778db>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noNewLine'].str.replace('[^\w\s]','')
<ipython-input-29-fe888b4778db>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noPunctuation'].str.replac

<ipython-input-32-3b2f1c55a8d5>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nft_sales_df['DateTime'] = nft_sales_df.DateTime.str.split(' ').str[0]
<ipython-input-29-fe888b4778db>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet_noURL'] = df['Tweet_nospecial'].str.replace('http\S+','')
<ipython-input-29-fe888b4778db>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noNewLine'].str.replace('[^\w\s]','')
<ipython-input-29-fe888b4778db>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noPunctuation'].str.replac

<ipython-input-29-fe888b4778db>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet_noURL'] = df['Tweet_nospecial'].str.replace('http\S+','')
<ipython-input-29-fe888b4778db>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noNewLine'].str.replace('[^\w\s]','')
<ipython-input-29-fe888b4778db>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noPunctuation'].str.replace('[_]','')
<ipython-input-29-fe888b4778db>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noNumbers"] = df['Tweet_noPunctuation'].str.replace(r'[0-9]',' ')
<ipython-input-32-3b2f1c55a8d5>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

<ipython-input-32-3b2f1c55a8d5>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nft_sales_df['DateTime'] = nft_sales_df.DateTime.str.split(' ').str[0]
<ipython-input-29-fe888b4778db>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet_noURL'] = df['Tweet_nospecial'].str.replace('http\S+','')
<ipython-input-29-fe888b4778db>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noNewLine'].str.replace('[^\w\s]','')
<ipython-input-29-fe888b4778db>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noPunctuation'].str.replac

<ipython-input-32-3b2f1c55a8d5>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nft_sales_df['DateTime'] = nft_sales_df.DateTime.str.split(' ').str[0]
<ipython-input-29-fe888b4778db>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet_noURL'] = df['Tweet_nospecial'].str.replace('http\S+','')
<ipython-input-29-fe888b4778db>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noNewLine'].str.replace('[^\w\s]','')
<ipython-input-29-fe888b4778db>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noPunctuation'].str.replac

<ipython-input-29-fe888b4778db>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet_noURL'] = df['Tweet_nospecial'].str.replace('http\S+','')
<ipython-input-29-fe888b4778db>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noNewLine'].str.replace('[^\w\s]','')
<ipython-input-29-fe888b4778db>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noPunctuation'].str.replace('[_]','')
<ipython-input-29-fe888b4778db>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noNumbers"] = df['Tweet_noPunctuation'].str.replace(r'[0-9]',' ')
<ipython-input-32-3b2f1c55a8d5>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

<ipython-input-32-3b2f1c55a8d5>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nft_sales_df['DateTime'] = nft_sales_df.DateTime.str.split(' ').str[0]
<ipython-input-29-fe888b4778db>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Tweet_noURL'] = df['Tweet_nospecial'].str.replace('http\S+','')
<ipython-input-29-fe888b4778db>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noNewLine'].str.replace('[^\w\s]','')
<ipython-input-29-fe888b4778db>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Tweet_noPunctuation"] = df['Tweet_noPunctuation'].str.replac

In [35]:
finaldf.to_csv('../model_data.csv', index=False)

## LSTM

In [12]:
def modelLSTM(trainSize, merged_df): #trainSize = 0.8, merged_df is output of mergeTweetSales('Anonymice')
    training_set_index = int(trainSize*merged_df.shape[0])
    
    training_set = merged_df.iloc[:training_set_index, 0:4]
    test_set = merged_df.iloc[training_set_index:, 0:4]
    
    sentiment_feature = "previous_day_sentiment" 
    features = [sentiment_feature, "prices_scaled"]
    training_set = training_set[features].dropna()
    
    X_train = training_set[sentiment_feature]
    X_train = X_train.values
    
    X_test = test_set[sentiment_feature]
    X_test = X_test.values
    
    y_train = training_set["prices_scaled"]
    y_train = y_train.values
    
    y_test = test_set["prices_scaled"]
    y_test = y_test.values
    
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_test, y_test = np.array(X_test), np.array(y_test)
    X_train = np.reshape(X_train, (X_train.shape[0],1))
    X_test = np.reshape(X_test, (X_test.shape[0],1))
    
    np.random.seed(1234)
    tf.random.set_seed(1234)

    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
    model.add(Dropout(rate = 0.3))

    model.add(LSTM(units=50, return_sequences = True))
    model.add(Dropout(rate = 0.3))

    model.add(LSTM(units=50, return_sequences = True))
    model.add(Dropout(rate = 0.3))

    model.add(LSTM(units=50, return_sequences = False))
    model.add(Dropout(rate = 0.3))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    model.fit(X_train, y_train, epochs=150, batch_size=int(0.48*training_set_index), verbose=1)
    
    y_pred = model.predict(X_test) 
    rms = mean_squared_error(y_test, y_pred, squared=False)
    
    rms2 = mean_squared_error(y_test, np.repeat(np.mean(y_train), len(y_test)), squared=False)
    
    predictive = rms < rms2
    
    return y_pred, rms, rms2, predictive
    

In [13]:
#to go from predicted scaled prices to actual prices
#y_pred = scaler.inverse_transform(y_pred) 

In [14]:
merged_df

,created_at,final_vader_score,prices_scaled,previous_day_sentiment,trends,trends_previous_day,trends_two_days_prior,num_tweets,collection_name
0,2021-09-17,52.2935,5.555677,13.1212,0.17,0.34,0.00,79,Anonymice
1,2021-09-17,52.2935,5.555677,13.1212,0.17,0.34,0.00,79,Anonymice
2,2021-09-18,137.7344,2.190590,52.2935,0.38,0.17,0.34,80,Anonymice
3,2021-09-19,14.3810,7.842699,137.7344,0.19,0.38,0.17,103,Anonymice
4,2021-09-20,48.7795,2.627685,14.3810,0.66,0.19,0.38,55,Anonymice
...,...,...,...,...,...,...,...,...,...
3318,2022-01-25,0.0000,-0.290027,0.4767,0.10,0.00,0.07,1,Linksdao
3319,2022-01-27,0.0000,-0.140497,0.0000,0.12,0.00,0.10,2,Linksdao
3320,2022-01-28,0.9431,-0.259495,0.0000,0.06,0.12,0.00,2,Linksdao
3321,2022-01-29,0.9304,-0.311147,0.9431,0.07,0.06,0.12,2,Linksdao


In [15]:
t = modelLSTM(0.8, merged_df)

Epoch 1/150
3/3 [==============================] - 5s 12ms/step - loss: 1.9496
Epoch 2/150
3/3 [==============================] - 0s 12ms/step - loss: 1.9418
Epoch 3/150
3/3 [==============================] - 0s 12ms/step - loss: 1.9338
Epoch 4/150
3/3 [==============================] - 0s 12ms/step - loss: 1.9262
Epoch 5/150
3/3 [==============================] - 0s 12ms/step - loss: 1.9159
Epoch 6/150
3/3 [==============================] - 0s 12ms/step - loss: 1.9045
Epoch 7/150
3/3 [==============================] - 0s 12ms/step - loss: 1.8949
Epoch 8/150
3/3 [==============================] - 0s 12ms/step - loss: 1.8817
Epoch 9/150
3/3 [==============================] - 0s 12ms/step - loss: 1.8698
Epoch 10/150
3/3 [==============================] - 0s 12ms/step - loss: 1.8520
Epoch 11/150
3/3 [==============================] - 0s 11ms/step - loss: 1.8366
Epoch 12/150
3/3 [==============================] - 0s 12ms/step - loss: 1.8270
Epoch 13/150
3/3 [==============================]

In [16]:
y_pred_actual_sales

NameError: name 'y_pred_actual_sales' is not defined

In [18]:
sales_df = pd.read_csv('sales_data.csv')

In [22]:
# pip install ipynb


Note: you may need to restart the kernel to use updated packages.


In [19]:
list_rms = []
list_rms2 = []
list_predictive = []

for x in sales_df.NFTcollection.unique():
    print(x)
    if x != "Bastard Gan Punks V2" and x != 'Chill Frogs Nft' and x != '420.Game Green Pass' and x!= "0Xvampire":
        list_rms.append(modelLSTM(0.8, mergeTweetSales(x))[1])
        list_rms2.append(modelLSTM(0.8, mergeTweetSales(x))[2])
        list_predictive.append(modelLSTM(0.8, mergeTweetSales(x))[3])


Anonymice


NameError: name 'mergeTweetSales' is not defined

In [ ]:
from statistics import mean

print(mean(list_rms))

print(mean(list_rms2))

In [ ]:
#list_rms

In [ ]:
list_nft = []

for x in sales_df.NFTcollection.unique():
    if x != "Bastard Gan Punks V2" and x != 'Chill Frogs Nft' and x != '420.Game Green Pass' and x!= "0Xvampire":
        list_nft.append(x)

In [ ]:
#list_nft

In [ ]:
df_nft = pd.DataFrame (list_nft, columns = ['NFTcol'])
df_rms = pd.DataFrame (list_rms, columns = ['rms'])
df_rms2 = pd.DataFrame (list_rms2, columns = ['rms_baseline'])
df_predictive = pd.DataFrame (list_predictive, columns = ['predictive'])

df_final = pd.merge(df_nft, df_rms, left_index=True, right_index=True)
df_final = pd.merge(df_final, df_rms2, left_index=True, right_index=True)
df_final = pd.merge(df_final, df_predictive, left_index=True, right_index=True)

In [ ]:
df_final['difference'] = df_final['rms_baseline'] - df_final['rms']

In [ ]:
df_final = df_final.sort_values(by="difference", ascending = False)
df_final

In [ ]:
df_final[df_final['predictive']==True]

In [ ]:
df_final[df_final['predictive']==False]